In [ ]:
import pandas as pd
import time
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import os
import undetected_chromedriver as uc
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

#driver
class WebScrappingDriver:
    def __init__(self):
        self.driver = uc.Chrome(version_main=134)
        self.driver.execute_cdp_cmd('Network.setCacheDisabled', {'cacheDisabled': True})

    def get_page(self, link):
        self.driver.delete_all_cookies()
        self.driver.get(link)

    def get_page_source_soup(self):
        source = self.driver.page_source
        soup = BeautifulSoup(source, "html.parser")
        return soup

    #NEW
    def get_xpath_value(self, input_xpath_val):
        return self.driver.find_element(By.XPATH, input_xpath_val)

    def send_keys_to_text_box_by_name(self, id_name_par, text_to_send):
        self.textbox = self.driver.find_element(By.NAME, id_name_par)
        self.textbox.clear()
        self.textbox.send_keys(text_to_send)
        self.textbox.send_keys(Keys.ENTER)

    def click_By_xpath(self,xpath_value):
        self.driver.find_element(By.XPATH, xpath_value).click()

    def click_By_id(self,id_value):
        self.driver.find_element(By.ID, id_value).click()

    def quit_driver(self):
        self.driver.quit()

In [ ]:
result_df = pd.DataFrame()
result_df['Keyword'] = None
result_df['Link'] = None
result_df['Özel Durum'] = None #-y-s
result_df['Özel Durum swA'] = None #-y-s

In [ ]:
ozel_durumlar_dict = {"-y-s":"SLP", "-p-":"Product", "-x-c":"Kategori", "-x-b":"Marka"} #SLP, Product, özel durum, sonundaki 2 - ile ayrılan a ile ve v ile bitcek -> swA(daha yazılmadı), Kategori, Marka

In [ ]:
kac_sf_aratilcak = 5

In [ ]:
magaza = "trendyol" #pazarama-trendyol
ek_magaza = " " + magaza
magaza_link = "www." + magaza + ".com" 

In [ ]:
"""
#TY SATICILARI ARATMAK İSTEDİĞİMDE; BUNU TÜM SERVERLARA KOPYALA
kac_sf_aratilcak = 1

magaza = "" 
ek_magaza = "" + magaza
magaza_link = "www."
"""

In [ ]:
scraper = WebScrappingDriver()
link = "https://www.google.com/"
scraper.get_page(link)
#-

In [ ]:
# Google Arama Sonucu Seçicileri
def return_each_google_search_info(soup_par):
    results_dict = {}
    for result in soup_par.find_all("div", class_="tF2Cxc"):  # Google sonuçlarını seçer
        try:
            url = result.find("span",{"jsaction":"rcuQ6b:npT2md;PYDNKe:bLV6Bd;mLt3mc"}).find("a")['href'] # URL
        except:
            url = None

        try:
            title = result.find("h3").text.strip() # Başlık
        except:
            title = "Bulunamadı"

        try:
            description = result.find("div",{"data-snf":"nke7rc"}).find('span').text # Açıklama 1
        except:
            description = "Bulunamadı"

        try:
            description2 = result.find("div",{"data-snf":"mCCBcf"}).text.replace("\xa0"," ").replace("  "," ").strip() # Açıklama 2
        except:
            description2 = "Bulunamadı"
        if url:
            results_dict[url] = [title, description, description2]
    return results_dict

In [ ]:
input_excel = pd.read_excel("detaylı search input/" + os.listdir("detaylı search input/")[0])
keywords_list = input_excel['Keywords'].values.tolist()
#-
result_df_sayac = 0
#-

In [ ]:
for kw_index, keyword in enumerate(keywords_list):
    trendyol_links_main_dict = {}

    key = 2
    while(key):
        try:
            keyword_to_search = str(keyword).lower() + ek_magaza
            scraper.send_keys_to_text_box_by_name('q', keyword_to_search) #aratıyor, enter denmesi lazım
            time.sleep(random.randint(22,36))#time.sleep(random.randint(18,32))
            #-
            soup = scraper.get_page_source_soup()

            all_search_tags = soup.find("div",{"id":"search"})
            
            if(all_search_tags):
                result_dict = return_each_google_search_info(soup)
                if result_dict:
                    trendyol_links_main_dict.update(result_dict)
                #---

                #Get Next Pages
                for next_page_count in range(2, kac_sf_aratilcak+1):
                    next_page_link = soup.find("a",{"aria-label":"Page " + str(next_page_count)})
                    if(next_page_link):
                        next_page_link_full = "https://www.google.com/" + next_page_link['href']
                        scraper.get_page(next_page_link_full)
                        time.sleep(random.randint(22,36))#time.sleep(random.randint(18,32))

                        soup = scraper.get_page_source_soup()
                        all_search_tags = soup.find("div",{"id":"search"})

                        if(all_search_tags):
                            result_dict = return_each_google_search_info(soup)
                            if result_dict:
                                trendyol_links_main_dict.update(result_dict)
                key = 0
        except:
            if(key):
                key -= 1
                scraper.quit_driver()
                time.sleep(5)
                print("hata verdi 10 saniye bekleyip aç kapa yapacak!")
                scraper = WebScrappingDriver()
                scraper.get_page(link)

    if trendyol_links_main_dict:
        for uniq_trendyol_link in trendyol_links_main_dict.keys():
            if magaza_link in uniq_trendyol_link and "text=" not in uniq_trendyol_link: #trendyol linki ise
                result_df.at[result_df_sayac, 'Keyword'] = keyword_to_search
                result_df.at[result_df_sayac, 'Link'] = uniq_trendyol_link
                result_df.at[result_df_sayac, 'Başlık'] = trendyol_links_main_dict[uniq_trendyol_link][0] #Başlık
                result_df.at[result_df_sayac, 'Açıklama 1'] = trendyol_links_main_dict[uniq_trendyol_link][1] #Açıklama 1
                result_df.at[result_df_sayac, 'Açıklama 2'] = trendyol_links_main_dict[uniq_trendyol_link][2] #Açıklama 2
                print(kw_index, uniq_trendyol_link)

                try:
                    splitted_links = uniq_trendyol_link.split("-")
                    for index, link_piece in enumerate(splitted_links):
                        if(len(link_piece) > 1):
                            if((link_piece[0] == "a") and (link_piece[1].isdigit())):
                                if((splitted_links[index+1][0] == "v") and (splitted_links[index+1][1].isdigit())):
                                    result_df.at[result_df_sayac, 'Özel Durum swA'] =  "swA"
                except:
                    pass
                for ozel_durum in ozel_durumlar_dict.keys():
                    if(ozel_durum in uniq_trendyol_link):
                        result_df.at[result_df_sayac, 'Özel Durum'] = ozel_durumlar_dict[ozel_durum]
                result_df_sayac += 1

    scraper.quit_driver()
    print("100kw arattı 30dk sonra tekrar aratılacak!")
    time.sleep(1)
    scraper = WebScrappingDriver()
    scraper.get_page(link)


    #100kw den sonra taraycıyı kapat, 30dk beklet, sonra tekrar başlat
    if str(kw_index)[-2:] == "50" or str(kw_index)[-2:] == "00":
        with pd.ExcelWriter("detaylı search output/" + os.listdir('detaylı search input/')[0].replace(".xlsx","_output.xlsx"), engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
            result_df.to_excel(writer,sheet_name="sheetName", index=False)

        scraper.quit_driver()
        print("100kw arattı 30dk sonra tekrar aratılacak!")
        time.sleep(3)
        scraper = WebScrappingDriver()
        scraper.get_page(link)

with pd.ExcelWriter("detaylı search output/" + os.listdir('detaylı search input/')[0].replace(".xlsx","_output.xlsx"), engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
    result_df.to_excel(writer,sheet_name="sheetName", index=False)

In [ ]:
with pd.ExcelWriter("detaylı search output/" + os.listdir('detaylı search input/')[0].replace(".xlsx","_output.xlsx"), engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
    result_df.to_excel(writer,sheet_name="sheetName", index=False)

In [ ]:
#GERİ KALAN 300 KÜSÜR BURAYA YAZILIP BAŞLATILACAK

In [ ]:
result_df

In [ ]:
scraper.quit_driver()